# Onboarding Challenge Part 2 🎮

In this notebook, we will use the [shivamshekhar's Chrome-T-Rex-Rush](https://github.com/shivamshekhar/Chrome-T-Rex-Rush). This is a python reimplementation of the popular Chrome's T-Rex runner game. Our goal is to hack it to make the T-Rex jump when our eye blinks!

In order to achieve this, we will have to acquire our signal in real time, detect when an eye blink occurs and then make the dinosaur jump.

In [ ]:
import os
from utils.utils import acquire_eeg
from muselsl import stream, list_muses
from multiprocessing import Process
from threading import Thread
from pynput.keyboard import Key, Controller

## Acquire the signal

The first step will be to acquire our data in real time. To achieve this, we will use the [Alexandre Barachant's muselsl library](https://github.com/alexandrebarachant/muse-lsl) and use the `acquire_eeg` function of our own (see the utils package of this repo). We will only need to write a callback function that receives all the samples via LSL.

In [ ]:
muses = list_muses()
if len(muses):
    stream_process = Process(target=stream, args=(muses[0]['address'],))
    stream_process.start()

## Analyze the signal
In the first notebook, we tested three way of setting an amplitude threshold. They all gave more or less the same results. This is why we are going to implement the simplest method of them all. We will use the same threshold as with the first method used in the first part. To test this real-time implementation, each time a blink is detected, we will print `Blink!`.

If it always print `Blink!`, test a greater threshold. If it never prints it, use a smaller value. You can also try to use another channel.

In [ ]:
# Adjust the threshold according to your own previous recordings
THRESHOLD = 100
ANALYZED_CHANNEL = 'TP10'
ACQUISITION_TIME = 20
SAMPLES_PER_CALLBACK = 1

In [ ]:
def eeg_callback(timestamps, eeg_data):
    if abs(eeg_data[ANALYZED_CHANNEL][0]) > THRESHOLD:
        print("Blink!")
    
acquire_eeg(ACQUISITION_TIME, callback=eeg_callback, eeg_chunck=SAMPLES_PER_CALLBACK)

## Connect the signal to the game input
We are now ready to play our game. To do so, we simply need to redefine our `eeg_callback` function. Instead of printing a blink message, we will simulate a press on the spacebar when the value of our signal is greater than our threshold. We will then start the game in the main process (the game cannot be start in a subprocess without crashing).

**Be careful** to stay on the game window since we are simulating space key presses.

In [ ]:
ANALYZED_CHANNEL = 'TP10'
ACQUISITION_TIME = 120
SAMPLES_PER_CALLBACK = 1

keyboard = Controller()

def eeg_callback(timestamps, eeg_data):
    if abs(eeg_data[ANALYZED_CHANNEL][0]) > THRESHOLD:
        keyboard.press(Key.space)
    else:
        keyboard.release(Key.space)

blink_eye_keyboard_thread = Thread(target=acquire_eeg, args=(ACQUISITION_TIME,), kwargs={'callback': eeg_callback, 'eeg_chunck': SAMPLES_PER_CALLBACK})
blink_eye_keyboard_thread.start()

os.chdir('./Chrome-T-Rex-Rush/')  # Workaround since it is not possible to import a folder with dash character.
import main

Now you just need to share your threshold value and your highscore!